# Exercise 3 - Range proofs with ZKPs

Now that we can do a privacy-preserving unlinkable selective disclosure, we can tackle the next step:
removing even more information by only proving the actually relevant information.
For an age check, the verifier doesn't need to know our birthdate.
It's enough if we can convince them that we are above, below, or in the range of a certain age.

## Sections

1. Issuer - Setting up and signing a credential
2. Creating a range proof using LegoGroth16
3. Creating a range proof using Bulletproofs++
4. Discussion: are we anonymous and unlinkable now?
5. Coding exercise - Using the ZKPs
6. Hard Coding Exercise - Revocation test: add an accumlator using a negative membership check

---

## 1. Issuer - Setting up and signing a credential

The first step of the issuer is the same as for the selective disclosure in exercise 2.

In [ ]:
// Defining the schema and one credential.
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        timeOfBirth: { type: 'integer', minimum: 0},
        height: {type: 'integer', minimum: 130, maximum: 210},
        weight: {type: 'number', minimum: 40, multipleOf: 0.1}
    }
}

const E_ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: new Date("1993-08-01T00:00:00").getTime(),
    profession: "IT Manager",
    height: 176,
    weight: 84
}

In [ ]:
import { initializeWasm, BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES,
        CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();
const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

// Generating a keypair for the issuer.
// We suppose that there is a PKI available which lets the verifier
// know the public key of the issuer.
const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));
const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

// Issuing a credential based on the E-ID data based on the specified schema,
// and sign it using the BBS+ library.
const baseSchema = CredentialSchema.essential();
baseSchema.properties[SUBJECT_STR] = E_ID_SCHEMA;  
const eIDSchema = new CredentialSchema(baseSchema);

const builder = new BBSCredentialBuilder();
builder.schema = eIDSchema;
builder.subject = E_ID_DATA;
const credential = builder.sign(secretKey);

// This is the data which is sent to the holder.
const credentialToSendToHolder = credential.toJSON();
console.log("This data is sent to the holder:", credentialToSendToHolder);

### Exercises

1. Change the content of the credential
2. Why is the `timeOfBirth` not a string like "21st of March 1998"?
3. Add a new field to the credential, but don't forget to also add it to the schema

---

## 2. Creating a range proof using LegoGroth16

Our first range proof is done using [LegoGroth16](https://eprint.iacr.org/2019/142)

### Verifier

We suppose that the holder takes contact with a verifier to get a service from it.
As a first step, the verifier must perform a trusted setup.
It consists of the following parts:

- a proving key, used to create the proof, and sent to the holder
- a verifying key, used to verify the proof

In [ ]:
// The verifier is responsible for creating these keys, then sharing only the proving key with 
// the credential holder.
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

### Credential Holder

With the proving key, the holder can now create a proof.
In our case, the holder reveals two attributes and creates a range proof to prove that: `170 <= height < 190`.

In [ ]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'

// The holder creates a proof for their height, as requested from the verifier.
// Instead of signing it, they call `finalize` to create the proof.
// The use-case is left as an exercise to the reader.
// My oldest daughter once convinced my youngest daughter that there is a minimal height
// for admission to UNIL, to avoid people signing up their dogs.
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
// Create a proof that `170 <= height < 190`
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', snarkProvingKey);
const presentation = builder.finalize();

// This is a serialized version, to be sent across an API to the verifier.
const lgProofForVerifier = presentation.toJSON();
console.log("Data sent to verifier:", lgProofForVerifier);

### Sending data to verifier

The following data is now sent to the verifier:

1. Selected data to be revealed
2. Schema of the credential
3. Proof of signature

All this information and more is stored in the "presentation object". Note that this object is a completely serializable JSON object.

### Verifier

The verifier can now make sure that the proof is correct and matches the public key of the issuer. 

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

// Now the verifier can check whether the proof is valid.
const predicateParams = new Map([['heightRangeCheck', snarkVerifyingKey]]);
const recreatedPres = Presentation.fromJSON(lgProofForVerifier)
console.log("Credential schema and revealed attributes:", lgProofForVerifier["spec"]["credentials"][0]);
console.log("Verification succeeded:", recreatedPres.verify([publicKey], undefined, predicateParams).verified);

### Exercises

1. What happens if you try to create a proof which is not satisfied by the credential? Why?
2. How does the verifier know the bounds check performed? Does it need this information? Why (not)?
3. Change the proof to be on a different field of the credential
4. Add a second range proof

---

## 3. Creating a range proof using Bulletproofs++

The docknetwork/crypto library offers different proof systems to create the range proofs.
While the LegoGroth16 is based on the well-known and tested Groth16 system, Bulletproofs(++) are somewhat newer, but
as we'll see in the measurement exercise, much faster and smaller.

### Holder

This is very similar to the creation of a proof with LegoGroth16, with two main differences:

- the setup is done on both the holder and the verifier side, using a "Common Reference String"
- these parameters are used instead of the ones created by the verifier in the case of LegoGroth16

In [ ]:
import { BoundCheckBppParams, PresentationBuilder } from '@docknetwork/crypto-wasm-ts'

// The holder creates a proof for their height, as requested from the verifier.
// Instead of signing it, they call `finalize` to create the proof.
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
const boundCheckBppParams1 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', boundCheckBppParams1);
const presentation = builder.finalize();

// This is a serialized version, to be sent across an API to the verifier.
const bpProofForVerifier = presentation.toJSON()
console.log("Data sent to the verifier:", bpProofForVerifier);

### Verifier

For Bulletproofs++, the verifier needs to recreate the same setup parameters as the prover (holder).
Only the "Common Reference String" must be the same.
This string can be sent from the verifier at the beginning of the 

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

// The verifier now proceeds to check the proof to see if it's valid.
const boundCheckBppParams2 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();
const predicateParams1 = new Map([['heightRangeCheck', boundCheckBppParams2]]);
const recreatedPres = Presentation.fromJSON(bpProofForVerifier);
console.log("Revealed attributes:", recreatedPres.spec.credentials[0].revealedAttributes);
console.log("Verification succeeded:", recreatedPres.verify([publicKey], undefined, predicateParams1).verified);

### Exercises

1. If you do the same exercises as for the LegoGroth16, what is the difference in the code?
2. What other difference do you experience when running the code?
3. Can you find how the verifier can be sure what is proven by the holder?

---

## 4. Discussion: are we anonymous and unlinkable now?

1. What did we gain by using Zero Knowledge Proofs?
2. What does the verifier learn about our credential?
3. Does this make the proof anonymous? Why? Why not?
4. Is the proof unlinkable? When is it? When isn't it?

---

## 5. Coding exercise - Using the ZKPs

Here are some more coding exercises to help you understand the code.
We propose you to copy the exercise into a new notebook, so you always have the original code ready for comparisons.

1. Create a proof that your age is in a certain range. Why is the age format given like this?
2. Add a new field for the postal code to the credential and to the schema. Now create a proof that you live in the canton de Vaud.

---

## 6. Hard Coding Exercise - Revocation test: add an accumlator using a negative membership check

Implement a revocation test of the credential using an accumulator.

This exercise is very hard, as unfortunately the docknetwork/crypto library is lacking a good documentation.
A good starting point is the following test:
[presentation-status-kb-accumulator.spec.ts](https://github.com/docknetwork/crypto-wasm-ts/blob/c32072b85150b6c1febadb29d2b8b7f4dbe7e40b/tests/anonymous-credentials/presentation-status-kb-accumulator.spec.ts#L143).
If you want to dive into this, please keep in mind:

- The tests in the `crypto-wasm-ts` library are written to work with different proving systems in mind.
  So they have a lots of variables which are filled out by other parts of the tests to make it as universal as possible.
- My preferred way to follow around the code is using VisualStudio, and then use `<CMD+CLICK>` to follow methods, `<CMD+OPTION+LEFT>`
  to go back

An accumulator is a cryptographic tool which allows to do (negative) membership proofs.
Instead of simply listing a set of numbers, an accumulator compresses these numbers using cryptographic algorithms.
Different accumulators exist with different trade-ofs:

- the size of the accumulator, depending on the number of elements in it
- whether it's possible to add and remove elements once the accumulator is set up
- if you can prove the existence and/or the absence of an element in the accumulator

Using the docknetwork/crypto library, you can add an inclusion or an absence proof using accumulators.
In order to prove the validity of the credential, the following needs to be done:

1. The issuer holds the accumulator which contains all revoked elements
2. If a credential is revoked, its ID needs to be stored in the accumulator
3. The holder creates an additional proof that their ID is not in the accumulator
4. When the verifier checks the proof, it can be sure that the credential of the holder hasn't been revoked yet

From a programming point of view, you have to add the following:

- Issuer
  - Setup the accumulator using something like [setupKBUniAccumulator](https://github.com/docknetwork/crypto-wasm-ts/blob/c32072b85150b6c1febadb29d2b8b7f4dbe7e40b/tests/anonymous-credentials/utils.ts#L824)
    - the `state` is not necessary
    - instead of adding 100 IDs, start with adding only two possible IDs
  - use `builder.setCredentialStatus` with a non-member check, like [here](https://github.com/docknetwork/crypto-wasm-ts/blob/c32072b85150b6c1febadb29d2b8b7f4dbe7e40b/tests/anonymous-credentials/presentation-status-kb-accumulator.spec.ts#L97)
    - the `registryID` can be any string
    - the `memberValue` needs to be the encoded user-ID from the `schema.encoder.encodeMessage` call
    - the `extra` field with the `blockNo` is optional and can be ignored.
      It is used to transfer additional information from the holder to the verifier.
- Holder
  - call `builder.addAccumInfoForCredStatus` with the values from the issuer. These values are public.
- Verifier
  - add a map with the public key of the accumulator to the call of `recreatedPres.verify`

Some questions:

- What security did we imply with this setup?
- What can go wrong?
- How can the verifier be sure that it didn't get a stale proof?
- How can the verifier be sure that the proof hasn't been done on a stale accumulator?